In [1]:
from model_plus import NewNet
from PIL import Image
import torch
from torchvision.transforms import Compose, Grayscale, ToTensor
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time

%matplotlib inline

# FPS

In [2]:
Threshold = 0.008
Class = '6'

# 指定网络
model = NewNet()
# 指定模型
model.load_state_dict(torch.load(f'./epoch20_DAGM_model/DAGM{Class}_best_model.pt'))
# 验证模式
model.eval()
# 指定GPU进行运算
model = model.to('cuda')
# 指定图片
imgpath = f"./image/Class{Class}/img1.PNG"
# 载入灰度图
img = Image.open(imgpath).convert('L')
# 以tensor格式的灰度图模式进行验证
transform = Compose([Grayscale(), ToTensor()])

# 传入图像
X = transform(img).to('cuda').unsqueeze(0)

In [32]:
LoopNumber = 26
# 计时
start_time = time.perf_counter()

for i in range(LoopNumber):
    EDX = model(X)
    S = torch.abs(X[0][0]-EDX[0][0])
    S_threshold = S.detach().cpu().numpy()
    
    ret, thresh_basic = cv2.threshold(S_threshold, Threshold, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((3, 3), np.uint8)
    img_erosion = cv2.dilate(thresh_basic, kernel, iterations=1)
    
    img_median = cv2.medianBlur(img_erosion, 5)
    
    edged = cv2.Canny((img_median*255).astype(np.uint8),1,0)
    
    img_original = X.detach().cpu().numpy()[0][0]
    imageOri = cv2.cvtColor(img_original, cv2.COLOR_GRAY2BGR)
    imageOri = np.power(imageOri, 3)
    
    thresh_inv_uint8 = img_median.astype(np.uint8)
    contours, h = cv2.findContours(thresh_inv_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    highlighted_img = cv2.drawContours(imageOri, contours, -1, (255, 0, 0), 3)

end_time = time.perf_counter()

print(f"DAGM{Class}经过{LoopNumber}次运算，花费{((time.perf_counter() - start_time)*1000)}ms")

DAGM6经过26次运算，花费996.8277999996644ms
